# Dynamics lineshapes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.kinematics.phasespace import Kallen
from ampform_dpd.dynamics import (
    BlattWeisskopf,
    BreitWignerMinL,
    BuggBreitWigner,
    EnergyDependentWidth,
    FlattéSWave,
    P,
    Q,
)
from ampform_dpd.io import cached

from polarimetry.io import display_doit, display_latex
from polarimetry.lhcb import load_three_body_decay
from polarimetry.lhcb.dynamics import formulate_flatte_1405
from polarimetry.lhcb.particle import Σ, K, load_particles, p, π

In [ ]:
z = sp.Symbol("z", positive=True)
L = sp.Symbol("L", integer=True, nonnegative=True)
display_doit(BlattWeisskopf(z, L))

In [ ]:
x, y, z = sp.symbols("x:z")
display_doit(Kallen(x, y, z))

In [ ]:
s, m0, mi, mj, mk = sp.symbols("s m0 m_i:k", nonnegative=True)
display_doit(P(s, mi, mj))
display_doit(Q(s, m0, mk))

In [ ]:
R = sp.Symbol("R")
l_R = sp.Symbol("l_R", integer=True, positive=True)
m, Γ0, m1, m2 = sp.symbols("m Γ0 m1 m2", nonnegative=True)
display_doit(EnergyDependentWidth(s, m, Γ0, m1, m2, l_R, R))

## Relativistic Breit-Wigner

In [ ]:
m_top, m_spec = sp.symbols(R"m_\mathrm{top} m_\mathrm{spectator}")
R_dec, R_prod = sp.symbols(R"R_\mathrm{res} R_{\Lambda_c}")
l_Λc = sp.Symbol(R"l_{\Lambda_c}", integer=True, positive=True)
display_doit(BreitWignerMinL(s, m_top, m_spec, m, Γ0, m1, m2, l_R, l_Λc, R_dec, R_prod))

## Bugg Breit-Wigner

In [ ]:
mKπ, m0, Γ0, mK, mπ, γ = sp.symbols(R"m_{K\pi} m0 Gamma0 m_K m_pi gamma")
bugg = BuggBreitWigner(mKπ**2, m0, Γ0, mK, mπ, γ)
q = P(mKπ**2, mK, mπ)
s_A = sp.Symbol("s_A")
definitions = {
    s_A: mK**2 - mπ**2 / 2,
    q: q.evaluate(),
}
display_latex({bugg: bugg.evaluate().subs({v: k for k, v in definitions.items()})})
display_latex(definitions)

One of the models uses a Bugg Breit-Wigner with an exponential factor:

In [ ]:
q = Q(s, m0, m1)
alpha = sp.Symbol("alpha")
bugg * sp.exp(-alpha * q**2)

## Flatté for S-waves

In [ ]:
Γ1, Γ2, m1, m2, mπ, mΣ = sp.symbols("Gamma1 Gamma2 m1 m2 m_pi m_Sigma")
display_doit(FlattéSWave(s, m, (Γ1, Γ2), (m1, m2), (mπ, mΣ)))

where, in this analysis, we couple the $\Lambda(1405)$ resonance to the channel $\Lambda(1405) \to \Sigma^-\pi^+$.

## Get decay chain

In [ ]:
particles = load_particles("../../data/particle-definitions.yaml")

In [ ]:
decay = load_three_body_decay(["L(1405)"], particles, min_ls=False)

In [ ]:
for i, chain in enumerate(decay.chains):
    print(i, chain.incoming_ls, chain.outgoing_ls)

In [ ]:
extracted_expr_chain0, params_chain0 = formulate_flatte_1405(decay.chains[0])
extracted_expr_chain1, params_chain1 = formulate_flatte_1405(decay.chains[1])

In [ ]:
extracted_expr_chain1

In [ ]:
flatte_function_chain0 = cached.lambdify(
    extracted_expr_chain0.doit(),
    parameters=params_chain0,
    backend="jax",
)
flatte_function_chain1 = cached.lambdify(
    extracted_expr_chain1.doit(),
    parameters=params_chain1,
    backend="jax",
)

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(9, 9))
ax0, ax1 = axes
x = np.linspace(0, 5, 300)
ax0.plot(x, np.abs(flatte_function_chain0({"sigma2": x**2})), label=R"Abs $F$")
ax0.plot(x, flatte_function_chain0({"sigma2": x**2}).imag, label=R"Im $F$")
ax0.plot(x, flatte_function_chain0({"sigma2": x**2}).real, label=R"Re $F$")
ax0.axvline(
    K.mass + p.mass,
    color="gray",
    linestyle="dotted",
    label=R"$K p$",
)
ax0.axvline(
    Σ.mass + π.mass,
    color="black",
    linestyle="--",
    label=R"$\Sigma \pi$",
)
ax0.axvline(
    flatte_function_chain0.parameters["m_{L(1405)}"],
    color="red",
    linestyle="--",
    label=R"Mass $m_{L(1405)}$",
)
ax0.legend()

ax1.plot(x, np.abs(flatte_function_chain1({"sigma2": x**2})), label=R"Abs $F$")
ax1.plot(x, flatte_function_chain1({"sigma2": x**2}).imag, label=R"Im $F$")
ax1.plot(x, flatte_function_chain1({"sigma2": x**2}).real, label=R"Re $F$")
ax1.axvline(
    K.mass + p.mass,
    color="gray",
    linestyle="dotted",
    label=R"$K p$",
)
ax1.axvline(
    Σ.mass + π.mass,
    color="black",
    linestyle="--",
    label=R"$\Sigma \pi$",
)
ax1.axvline(
    flatte_function_chain0.parameters["m_{L(1405)}"],
    color="red",
    linestyle="--",
    label=R"Mass $m_{L(1405)}$",
)
ax1.set_title(
    f"Decay chain 1, production {decay.chain[1].incoming_ls}, decay {chain[1].outgoing_ls}"
)
ax1.legend()
plt.show()